# Gif Dataset #

In [2]:
"""
PyTorch Giff Dataset Class for loading gifs using Pytorch DataLoader.
This Dataset resizes all frames to a specific size.

Facundo Calcagno
"""
from __future__ import print_function, division
import cv2
import os
import torch
import numpy as np
import pickle
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image,ImageFilter , ImageOps, ImageDraw
import imageio
import pandas as pd
import torch.utils.data as data

In [3]:
class gifDataSet(data.Dataset):
    """Dataset Class for loading Gif into a 3D Tensor"""
    def __init__(self,gifList,rootDir, channels,  timeDepth, xSize, ySize, 
                 startFrame,endFrame,transform=None):
        """
        Args:
        clipsList (string): Path to the clipsList file with labels.
        rootDir (string): Directory with all the gifs.
        transform (callable, optional): Optional transform to be applied
            on a sample.
        channels: Number of channels of frames
        timeDepth: Number of frames to be loaded in a sample
        xSize, ySize: Dimensions of the frames
        mean: Mean valuse of the training set videos over each channel
        """
        self.gifList = gifList
        self.rootDir = rootDir
        self.channels = channels
        self.timeDepth = timeDepth
        self.xSize = xSize
        self.ySize = ySize
        #self.mean = mean
        self.transform = transform
        self.startFrame=startFrame
        self.endFrame=endFrame

    
    def __len__(self):
        return len(self.gifList)
    
    def crop6(self,im):
        number_of_cols=3
        W=im.width
        H=im.height
        w=(W-16)/3
        h=(H-24)/2
        images=[]
        w1=0
        w2=w
        for i in range(number_of_cols):
            im1=im.crop((w1, 0, w2, h))
            images.append(im1)
            im1=im.crop((w1, h+8, w2, 2*h+8))
            images.append(im1)
            w1=w2+8
            w2=w2+w+8
        return images
    
    def readGif(self, gifFile):
        # Open the gif file, crop it and return the frames in a tensor
        image_gif=Image.open(self.rootDir+ gifFile, mode='r')
        frames = torch.FloatTensor(self.timeDepth,  self.xSize, self.ySize,self.channels)
        nframes = 0
        nframesin=0
        while image_gif:
            if self.startFrame<nframes<self.endFrame:
                six_images=self.crop6(image_gif)
                # I keep image 4 that is the original, but in the future I might use others 
                # as different channels
                if self.channels == 3: pil_image = six_images[4].convert("RGB")
                if self.channels == 1: pil_image = six_images[4].convert("L")  
                imResize=pil_image.resize((self.xSize, self.ySize),Image.ANTIALIAS)
                frame = torch.from_numpy(np.array(imResize))
                if self.channels == 1: frame=torch.unsqueeze(frame,2)
                #frame = frame.permute(2, 0, 1)
                frames[nframesin, :, :, :] = frame
                nframesin+=1
            nframes += 1
            try:
                image_gif.seek( nframes )
            except EOFError:
                break;
            
        image_gif.close()
        return frames
    
    def makeGif(self,gifTensor,gifFile):
        # With a Pytorch Tensor generate a gif file
        with imageio.get_writer(gifFile, mode='I') as writer:
            nframes=0
            while nframes<self.timeDepth:
                frame=gifTensor[nframes,:,:,:].numpy()
                writer.append_data(frame)
                nframes+=1
        return writer
    
    def __getitem__(self, idx):
        gifFile= os.path.join(self.rootDir, self.gifList[idx][0])
        clip = self.readGif(gifFile)
        if self.transform:
            clip = self.transform(clip)
        sample = {'clip': clip, 'label': self.gifList[idx][1]}
        

In [4]:
rootDir="gifs/"
import glob
mypath=rootDir+'Mediophyceae_Hemiaulus'
a=glob.glob(rootDir+'*.gif')

We are going to keep the top right image of all gifs and resize to a 100x100x3 tensor.

In [5]:
#gifListFile='gifs.pickle';rootDir="gifs/";channels=1;timeDepth=3;xSize=100;ySize=100
#startFrame=1;endFrame=5

In [6]:
#gifDataset= gifDataSet(a,rootDir,channels,timeDepth,xSize,ySize,startFrame,endFrame)

### Testing Reading and writing a Gif ##

### Original Image

<img src="gifs/10753468_S127--D0--R27--G100010824--A131203--L02236--animation.gif" width="300">

### Convertion Code ###

With the Following Code we are able to Input the full image, retreive the original Image from the gif, export a Pytorch Tensor and with that Tensor export a Gif file if necessary.

In [7]:
#tensor=gifDataset.readGif("10753468_S127--D0--R27--G100010824--A131203--L02236--animation.gif")

In [8]:
#image= gifDataset.makeGif(tensor,'gifs/test.gif')

### Converted Image ###

![alt text](gifs/test.gif "Conv 2D")

### Use train and test files created in ``Train Test text file``  to import files into the Class

In [9]:
trainset="plancton-train.csv"
testset="plancton-test.csv"
trainlist=pd.read_csv(trainset,delimiter=";",header=1,names =("file","folder","type"))
testlist=pd.read_csv(testset,delimiter=";",header=1,names =("file","folder","type"))

In [13]:
train_loader = data.DataLoader(gifDataSet(gifList=trainlist,
                                          rootDir="../TaraData/exportimg_160_20180423_0936/",
                                          channels=1,timeDepth=3,xSize=100,ySize=100,
                                          startFrame=1,endFrame=5), 
                               batch_size=1, shuffle=True)

In [14]:
test_loader = data.DataLoader(gifDataSet(gifList=testlist,
                                          rootDir="../TaraData/exportimg_160_20180423_0936/",
                                          channels=1,timeDepth=3,xSize=100,ySize=100,
                                          startFrame=1,endFrame=5), 
                               batch_size=1, shuffle=True)